In [76]:
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore.nn import Accuracy
from mindspore import Tensor, context
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor
from mindspore.dataset import vision, transforms
import mindspore.dataset as ds
from mindspore.common.initializer import XavierUniform
import pandas as pd
import numpy as np
import os
from PIL import Image
import mindspore.common.dtype as mstype

In [77]:
context.set_context( mode = context.GRAPH_MODE, device_target = "GPU" if mindspore.context.get_context( "device_target" ) == "GPU" else "CPU" )

In [78]:
class Bottleneck( nn.Cell ):
    def __init__( self, in_channels, mid_channels, out_channels, stride = 1 ):
        super( Bottleneck, self ).__init__()
        self.conv1 = nn.Conv2d( in_channels, mid_channels, kernel_size = 1, stride = 1, has_bias = False )
        self.bn1 = nn.BatchNorm2d( mid_channels )
        self.conv2 = nn.Conv2d( mid_channels, mid_channels, kernel_size=3, stride = stride, pad_mode = "pad", padding = 1, has_bias = False )
        self.bn2 = nn.BatchNorm2d( mid_channels )
        self.conv3 = nn.Conv2d( mid_channels, out_channels, kernel_size = 1, stride = 1, has_bias = False )
        self.bn3 = nn.BatchNorm2d( out_channels )
        self.relu = nn.ReLU()

        self.downsample = nn.SequentialCell()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.SequentialCell( [
                nn.Conv2d( in_channels, out_channels, kernel_size = 1, stride = stride, has_bias = False ),
                nn.BatchNorm2d( out_channels )
            ] )

    def construct( self, x ):
        residual = x
        out = self.conv1( x )
        out = self.bn1( out )
        out = self.relu( out )
        out = self.conv2( out )
        out = self.bn2( out )
        out = self.relu( out )
        out = self.conv3( out )
        out = self.bn3( out )
        residual = self.downsample( residual )
        out += residual
        out = self.relu( out )
        return out

In [79]:
class Net(nn.Cell):
    def __init__(self, num_classes=5):
        super(Net, self).__init__()
        self.features = nn.SequentialCell([
            nn.Conv2d( 3, 96, kernel_size = 11, stride = 4, pad_mode = "pad", padding = 2 ),
            nn.ReLU(),
            nn.MaxPool2d( kernel_size = 3, stride = 2 ),

            Bottleneck( 96, 32, 128 ),

            nn.Conv2d( 128, 256, kernel_size = 5, stride = 1, pad_mode = "pad", padding = 2 ),
            nn.ReLU(),
            nn.MaxPool2d( kernel_size = 3, stride = 2 ),

            Bottleneck( 256, 64, 512 ),

            nn.Conv2d( 512, 768, kernel_size = 3, stride = 1, pad_mode = "pad", padding = 1 ),
            nn.ReLU(),

            Bottleneck( 768, 128, 768 ),

            nn.Conv2d( 768, 1024, kernel_size = 3, stride = 1, pad_mode = "pad", padding = 1 ),
            nn.ReLU(),

            Bottleneck( 1024, 128, 1280 ),

            nn.AdaptiveAvgPool2d( ( 1, 1 ) ),
        ])
        
        self.classifier = nn.SequentialCell([
            nn.Dropout( p = 0.5 ),
            nn.Dense( 1280, 1024, weight_init = XavierUniform() ),
            nn.ReLU(),
            nn.Dropout( p = 0.5 ),
            nn.Dense( 1024, 512, weight_init = XavierUniform() ),
            nn.ReLU(),
            nn.Dense( 512, 256, weight_init = XavierUniform() ),
            nn.ReLU(),
            nn.Dense( 256, num_classes, weight_init = XavierUniform() )
        ])

    def construct( self, x ):
        x = self.features( x )
        x = x.view( x.shape[0], -1 )
        x = self.classifier( x )
        return x

In [80]:
class CassavaDataset:
    def __init__( self, csv_f, img_folder, transform, s_num = 6000 ):
        self.img_labels = pd.read_csv( csv_f )[:s_num]
        self.img_folder = img_folder
        self.transform = transform

    def __getitem__( self, idx ):
        img_name = str( self.img_labels.iloc[ idx, 0 ] )
        label = self.img_labels.iloc[ idx, 1 ]
        img_path = os.path.join( self.img_folder, img_name )
        image = Image.open( img_path ).convert( "RGB" )
        if self.transform:
            image = self.transform( image )
        return np.array( image, dtype = np.float32 ), np.array( label, dtype = np.int32 )

    def __len__(self):
        return len(self.img_labels)
    


def transform( img ):
    trans = transforms.Compose( [
        vision.Decode(),
        vision.Resize( ( 256, 256 ) ),
        vision.Rescale( 1.0 / 255.0, 0 ),
        vision.Normalize( mean = ( 0.485, 0.456, 0.406 ), std = ( 0.229, 0.224, 0.225 ) ),
        vision.HWC2CHW()
    ] )
    return trans( img )


def create_dataset(dataset_path, batch_size=32, train=True):
    data_set = ds.ImageFolderDataset(dataset_path, shuffle=train)

    
    type_cast_op = transforms.TypeCast(mstype.int32)
    data_set = data_set.map(input_columns="image", operations=transform)
    data_set = data_set.map(input_columns="label", operations=type_cast_op)
    if train:
        data_set = data_set.shuffle(buffer_size=1000)
    data_set = data_set.batch(batch_size, drop_remainder=True)
    
    return data_set


train_dataset_path = "../data/train_photos_14"
test_dataset_path = "../data/test_photos_14"
batch_size = 32

trainloader = create_dataset(train_dataset_path, batch_size=batch_size, train=True)
testloader = create_dataset(test_dataset_path, batch_size=batch_size, train=False)



net = Net( num_classes = 5 )
loss_fn = nn.SoftmaxCrossEntropyWithLogits( sparse = True, reduction = 'mean' )
optimizer = nn.Adam( params = net.trainable_params(), learning_rate = 0.01 )

net_with_loss = nn.WithLossCell( net, loss_fn )
train_net = nn.TrainOneStepCell( net_with_loss, optimizer )
train_net.set_train()

model = Model( net, loss_fn, optimizer, metrics={ 'accuracy': Accuracy() } )



epoch_num = 1
best_acc = 0.0
best_ckpt = "Net.ckpt"

for epoch in range( epoch_num ):
    model.train( 1, trainloader, callbacks = [ LossMonitor() ], dataset_sink_mode = False )

    acc = model.eval( testloader, dataset_sink_mode = False)[ "accuracy" ]
    print( f'[ epoch { epoch + 1 } ] train_loss: { LossMonitor() } test_accuracy: { acc:.3f }' )

    if acc > best_acc:
        best_acc = acc
        mindspore.save_checkpoint( net, best_ckpt )




epoch: 1 step: 1, loss is 2.372617483139038
epoch: 1 step: 2, loss is 35.480472564697266
epoch: 1 step: 3, loss is 4.451798439025879
epoch: 1 step: 4, loss is 1.8142179250717163
epoch: 1 step: 5, loss is 1.6171833276748657
epoch: 1 step: 6, loss is 1.5361323356628418
epoch: 1 step: 7, loss is 1.4608381986618042
epoch: 1 step: 8, loss is 1.5687873363494873
epoch: 1 step: 9, loss is 1.6121478080749512
epoch: 1 step: 10, loss is 1.6702176332473755
epoch: 1 step: 11, loss is 1.4999836683273315
epoch: 1 step: 12, loss is 1.2513704299926758
epoch: 1 step: 13, loss is 1.0561336278915405
epoch: 1 step: 14, loss is 2.2487494945526123
epoch: 1 step: 15, loss is 2.0726120471954346
epoch: 1 step: 16, loss is 1.2171597480773926
epoch: 1 step: 17, loss is 1.340877652168274
epoch: 1 step: 18, loss is 1.6367626190185547
epoch: 1 step: 19, loss is 1.507347583770752
epoch: 1 step: 20, loss is 1.662670373916626
epoch: 1 step: 21, loss is 1.5818997621536255
epoch: 1 step: 22, loss is 1.3940844535827637
ep

ValueError: Invalid format specifier

In [82]:
result = model.eval(testloader)
print(f"Test accuracy: {result['accuracy']}")

Test accuracy: 0.5192307692307693
